**Load Library**

In [2]:
import os
import spacy
import pandas as pd
import numpy as np
import geopandas as gpd
import re
import math
import string
import unicodedata
import gensim
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
from joblib import dump
from joblib import load
import contextily as ctx
import urllib.request

from scipy.spatial.distance import cdist

from shapely.geometry import Point

from sklearn.preprocessing import OneHotEncoder  # We don't use this but I point out where you *could*
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC


from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk import ngrams, FreqDist

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from gensim.matutils import Sparse2Corpus
from gensim.matutils import corpus2dense
from gensim.models import tfidfmodel
from gensim.models import Word2Vec
from gensim.models import TfidfModel
from gensim.models import KeyedVectors
from gensim.models.ldamodel import LdaModel

from bs4 import BeautifulSoup
from wordcloud import WordCloud, STOPWORDS

nltk.download('stopwords')
stopword_list = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SBH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# 从norm后的数据读取csv
Airbnb_Listing = pd.read_csv(os.path.join("..","Data",'Airbnb_Listing_norm.csv'))

# 假设 `texts` 是你的分词后的文本数据（每个文本为词的列表）
texts_word2vec = Airbnb_Listing['amenities']  # 你的分词数据

# 指定训练参数
dims = 400
window = 10

# 训练Word2Vec模型
model = Word2Vec(sentences=texts_word2vec, vector_size=dims, window=window, min_count=1, workers=4)


In [6]:
# 保存word2vec模型
model.save(os.path.join("Model",f"word2vec-d{dims}-w{window}.model"))

FileNotFoundError: [Errno 2] No such file or directory: 'Model\\word2vec-d400-w10.model'

读取数据源

In [ ]:
Airbnb_Listing_origin = pd.read_csv("./Data/Data_InsideAirbnb/listings.csv.gz")

计算所有listing的average incom


In [ ]:
# 每个listing的收入与average收入相比
if Airbnb_Listing_origin['price'].dtype == 'object':
    Airbnb_Listing_origin['price'] = Airbnb_Listing_origin['price'].str.replace('$', '').str.replace(',', '').astype(float)
Airbnb_Listing['sum_income'] = Airbnb_Listing_origin['minimum_nights']*2.7*Airbnb_Listing_origin['number_of_reviews_ltm']*Airbnb_Listing_origin['price']

average_income_forlisting = Airbnb_Listing['sum_income'].mean()
average_income_forlisting

In [ ]:
Airbnb_Listing['profitable'] = (Airbnb_Listing['sum_income'] >= average_income_forlisting).astype(int)

In [ ]:
# 转换文本向量
def document_vector(word2vec_model, doc):
    # 移除不在词汇表中的词
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    # 处理空文档的情况
    if len(doc) == 0:
        return np.zeros(word2vec_model.vector_size)
    # 计算均值向量
    return np.mean(word2vec_model.wv[doc], axis=0)

# 为每个文档计算向量
doc_vectors = np.array([document_vector(model, doc) for doc in texts])

分词处理：您使用的是 text.split(" ") 来分词。这意味着您假设文本中的每个单词之间由两个空格分隔。请确保这与您的数据格式一致。如果是普通英文文本，通常单词之间只有一个空格，那么应该使用 text.split()。

空文档处理：在 document_vector 函数中，如果文档中所有的词都不在模型的词汇表中，那么 word2vec_model.wv[doc] 将是一个空列表，这会导致 np.mean 报错。您需要处理这种情况。

文档向量计算：当您计算文档向量时，您使用的是 np.array([document_vector(model, doc) for doc in texts])。这里 texts 应该是分词后的文本数据。请确保 texts 和 texts_word2vec 是一致的，即 texts 应该是用于训练 Word2Vec 模型的相同数据。

标签和特征数据：确保 labels 是与 doc_vectors 对应的目标变量数组。labels 应该有与 doc_vectors 相同数量的元素。

模型性能评估：在最后，您计算了准确率，这是评估分类模型性能的一个常用指标。根据您的应用情况，可能还需要考虑其他指标，如精确率、召回率和F1分数。

异常和错误处理：在实际应用中，建议添加异常处理和错误检查，确保代码的健壮性。

In [ ]:
# 随机森林方法

#使用任何类型的分类器来预测是否income超过平均值
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(doc_vectors, Airbnb_Listing['profitable'], test_size=0.2, random_state=42)

# 训练分类器
classifier = RandomForestClassifier(n_estimators=300, random_state=42)
classifier.fit(X_train, y_train)

# 预测测试集
y_pred = classifier.predict(X_test)

# 评估模型
print("Accuracy:", accuracy_score(y_test, y_pred))


In [ ]:
#SVM方法
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(doc_vectors, Airbnb_Listing['profitable'], test_size=0.2, random_state=42)

# 创建 SVM 分类器实例
svm_classifier = SVC(random_state=42)

# 训练分类器
svm_classifier.fit(X_train, y_train)

# 预测测试集
y_pred = svm_classifier.predict(X_test)

# 评估模型
print("Accuracy:", accuracy_score(y_test, y_pred))



保存SVM模型与保存后的Airbnb_Listing_norm_income

In [ ]:
dump(svm_classifier, 'svm_classfier_model.joblib')